# Predict

In [ ]:
from predict import predict_image

predict_image(input_file='/media/eolika/BlueMindAI/CT/dataset/plethora_dataset/plethora-151/ct.nii.gz',
              output='/media/eolika/BlueMindAI/CT/dataset/plethora_dataset/plethora-151',
              model='/home/eolika/Desktop/BlueMind/nnUNet/trains/nnUNet_results/Dataset930_mosaic_heart/nnUNetTrainerDiceCELoss_noSmooth_50epochs__nnUNetPlans__3d_fullres',
              separate_masks=True)

### batch prediction

In [ ]:
import os
import natsort
from tqdm import tqdm
from predict import predict_image

dataset = "/media/eolika/BlueMindAI/CT/dataset/plethora_dataset"
output_mask_name = "total"
model='/home/eolika/Desktop/BlueMind/nnUNet/trains/nnUNet_results/nnUNetTrainerDiceCELoss_noSmooth_500epochs__nnUNetPlans__3d_fullres'

cases = natsort.natsorted([case for case in os.listdir(dataset) if os.path.isdir(os.path.join(dataset, case))])

pbar = tqdm(cases)

for case in pbar:

    pbar.set_description_str(case)

    ct_file_path = os.path.join(dataset, case, 'ct.nii.gz')
    output_dir = os.path.join(dataset, case, 'segmentations')

    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    predict_image(input_file=ct_file_path,
              output=f'{output_dir}/{output_mask_name}.nii.gz',
              model=model,
              nnunet_verbose=False)

### Get sub-masks from total mask

In [ ]:
import nibabel as nib
import numpy as np
from tqdm import tqdm
import natsort
import os

dataset = "/media/eolika/BlueMindAI/CT/dataset/plethora_dataset"
total_mask_name = 'total'

labels = {
    "bones": 1,
    "arteries": 2,
    "veins": 3,
    "muscles": 4,
    "spleen": 5,
    "kidney_right": 6,
    "kidney_left": 7,
    "gallbladder": 8,
    "liver": 9,
    "liver_vessels": 10,
    "stomach": 11,
    "pancreas": 12,
    "adrenal_gland_right": 13,
    "adrenal_gland_left": 14,
    "esophagus": 15,
    "brain": 16,
    "small_bowel": 17,
    "duodenum": 18,
    "colon": 19,
    "urinary_bladder": 20,
    "lung_upper_lobe_left": 21,
    "lung_lower_lobe_left": 22,
    "lung_upper_lobe_right": 23,
    "lung_middle_lobe_right": 24,
    "lung_lower_lobe_right": 25,
    "lung_trachea_bronchia": 26,
    "lung_vessels": 27,
    "heart_myocardium": 28,
    "heart_atrium_left": 29,
    "heart_ventricle_left": 30,
    "heart_atrium_right": 31,
    "heart_ventricle_right": 32,
    "liver_formation": 33,
    "lung_formation": 34,
    "kidney_formation": 35,
    "pleural_effusion": 36,
    "intracerebral_hemorrhage": 37
}

cases = natsort.natsorted([case for case in os.listdir(dataset) if os.path.isdir(os.path.join(dataset, case))])

pbar = tqdm(cases)

for case in pbar:

    pbar.set_description_str(case)

    ct_file_path = os.path.join(dataset, case, 'ct.nii.gz') 
    output_dir = os.path.join(dataset, case, 'segmentations')
    mask_file_path = os.path.join(output_dir, 'total.nii.gz') 

    ct_img = nib.load(ct_file_path)
    mask_img = nib.load(mask_file_path)
    mask_data = mask_img.get_fdata()

    for label, indx in labels.items():
        label_mask = np.array(mask_data == indx).astype(np.uint8)
        nib.save(nib.Nifti1Image(label_mask, mask_img.affine), os.path.join(output_dir, label+'.nii.gz'))